In [1]:
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense 
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras.optimizers import Adam
import numpy as np

In [2]:

class PolicyGradientNetwork(keras.Model):
    def __init__(self, n_actions, fc1_dims=256, fc2_dims=256):
        super(PolicyGradientNetwork, self).__init__()
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.n_actions = n_actions

        self.fc1 = Dense(self.fc1_dims, activation='relu')
        self.fc2 = Dense(self.fc2_dims, activation='relu')
        self.pi = Dense(n_actions, activation='softmax')

    def call(self, state):
        value = self.fc1(state)
        value = self.fc2(value)

        pi = self.pi(value)

        return pi

In [14]:

class Agent:
    def __init__(self, alpha=0.003, gamma=0.99, n_actions=4,
                 layer1_size=256, layer2_size=256):

        self.gamma = gamma
        self.lr = alpha
        self.n_actions = n_actions
        self.state_memory = []
        self.action_memory = []
        self.reward_memory = []
        self.policy = PolicyGradientNetwork(n_actions=n_actions)
        self.policy.compile(optimizer=Adam(learning_rate=self.lr))

    def choose_action(self, observation):
        state = tf.convert_to_tensor([observation],dtype=tf.float32)
        probs = self.policy(state)
        action_probs = tfp.distributions.Categorical(probs=probs)
        action = action_probs.sample()
        
        return action.numpy()[0]
    

    def store_transition(self, observation, action, reward):
        self.state_memory.append(observation)
        self.action_memory.append(action)
        self.reward_memory.append(reward)

    def learn(self):
        actions = tf.convert_to_tensor(self.action_memory, dtype=tf.float32)
        rewards = tf.convert_to_tensor(self.reward_memory)
        
        G = np.zeros_like(rewards)
        for t in range(len(rewards)):
            G_sum = 0
            discount = 1
            for k in range(t, len(rewards)):
                G_sum += rewards[k]*discount
                discount *= self.gamma
                
            G[t] = G_sum
            
        with tf.GradientTape() as tape:
            loss = 0
            for idx,(g , state) in enumerate(zip(G, self.state_memory)):
                state = tf.convert_to_tensor([observation],dtype=tf.float32)
                probs = self.policy(state)
                action_probs = tfp.distributions.Categorical(probs=probs)
                log_prob = action_probs.log_prob(actions[idx])
                loss += -g + tf.squeeze(log_prob)
                
        gradient = tape.gradient(loss, self.policy.trainable_variables)
        self.policy.optimizer.apply_gradients(zip(gradient,self.policy.trainable_variables))
        
        self.state_memory = []
        self.action_memory = []
        self.reward_memory = []



In [15]:
# !pip install mujoco_py==2.0.2.8
# !pip install 'gym[all]'
import gym

In [16]:
if __name__ == '__main__':
    agent = Agent(alpha=0.0005,  gamma=0.99,n_actions=4)

    env = gym.make('LunarLander-v2')
    score_history = []




episode:  0 score: -145.2 average score -145.2
episode:  1 score: -105.7 average score -125.4
episode:  2 score: -434.4 average score -228.4
episode:  3 score: -139.5 average score -206.2
episode:  4 score: -192.2 average score -203.4
episode:  5 score: -104.1 average score -186.8
episode:  6 score: -165.4 average score -183.8
episode:  7 score: -445.6 average score -216.5
episode:  8 score: -123.5 average score -206.2
episode:  9 score: -119.0 average score -197.4
episode:  10 score: -207.4 average score -198.4
episode:  11 score: -236.5 average score -201.5
episode:  12 score: -13.3 average score -187.1
episode:  13 score: -344.0 average score -198.3
episode:  14 score: -96.1 average score -191.5
episode:  15 score: -262.1 average score -195.9
episode:  16 score: -96.5 average score -190.0
episode:  17 score: -91.5 average score -184.6
episode:  18 score: -91.2 average score -179.6
episode:  19 score: -251.5 average score -183.2
episode:  20 score: -21.7 average score -175.5
episode:

episode:  171 score: -150.0 average score -158.3
episode:  172 score: -160.8 average score -157.6
episode:  173 score: -146.7 average score -157.2
episode:  174 score: -122.9 average score -156.8
episode:  175 score: -181.3 average score -155.7
episode:  176 score: -174.2 average score -154.9
episode:  177 score: -147.3 average score -155.1
episode:  178 score: -100.9 average score -154.0
episode:  179 score: -164.4 average score -154.7
episode:  180 score: -92.5 average score -154.1
episode:  181 score: -136.1 average score -154.0
episode:  182 score: -133.1 average score -153.8
episode:  183 score: -120.9 average score -153.5
episode:  184 score: -126.9 average score -152.8
episode:  185 score: -96.4 average score -152.1
episode:  186 score: -205.6 average score -154.2
episode:  187 score: -149.2 average score -154.5
episode:  188 score: -101.3 average score -153.6
episode:  189 score: -115.8 average score -152.9
episode:  190 score: -371.7 average score -153.8
episode:  191 score: -

episode:  339 score: -107.8 average score -143.1
episode:  340 score: -157.7 average score -143.2
episode:  341 score: -161.0 average score -143.0
episode:  342 score: -160.4 average score -143.2
episode:  343 score: -149.1 average score -143.2
episode:  344 score: -154.9 average score -142.8
episode:  345 score: -144.2 average score -142.9
episode:  346 score: -160.1 average score -143.8
episode:  347 score: -123.0 average score -144.0
episode:  348 score: -142.3 average score -143.9
episode:  349 score: -128.5 average score -141.3
episode:  350 score: -142.3 average score -142.2
episode:  351 score: -144.4 average score -142.5
episode:  352 score: -102.6 average score -142.0
episode:  353 score: -134.8 average score -142.3
episode:  354 score: -135.9 average score -142.3
episode:  355 score: -139.3 average score -142.0
episode:  356 score: -137.7 average score -141.8
episode:  357 score: -152.0 average score -141.9
episode:  358 score: -141.7 average score -139.8
episode:  359 score:

episode:  507 score: -154.9 average score -130.0
episode:  508 score: -115.0 average score -129.5
episode:  509 score: -149.4 average score -129.5
episode:  510 score: -104.6 average score -129.0
episode:  511 score: -136.1 average score -129.3
episode:  512 score: -105.5 average score -129.1
episode:  513 score: -123.2 average score -129.2
episode:  514 score: -136.6 average score -129.4
episode:  515 score: -138.9 average score -129.9
episode:  516 score: -146.2 average score -130.0
episode:  517 score: -161.4 average score -130.4
episode:  518 score: -82.2 average score -129.4
episode:  519 score: -86.5 average score -129.1
episode:  520 score: -151.7 average score -129.2
episode:  521 score: -152.7 average score -129.3
episode:  522 score: -132.1 average score -129.4
episode:  523 score: 4.0 average score -127.9
episode:  524 score: -117.4 average score -127.7
episode:  525 score: -131.4 average score -127.5
episode:  526 score: -49.5 average score -126.6
episode:  527 score: -108.

episode:  675 score: -108.2 average score -128.7
episode:  676 score: -140.9 average score -128.4
episode:  677 score: -116.9 average score -128.3
episode:  678 score: -142.5 average score -128.7
episode:  679 score: -156.4 average score -128.6
episode:  680 score: -107.7 average score -128.3
episode:  681 score: -146.8 average score -128.8
episode:  682 score: -128.9 average score -129.9
episode:  683 score: -109.2 average score -130.0
episode:  684 score: -133.8 average score -129.6
episode:  685 score: -139.7 average score -129.9
episode:  686 score: -148.3 average score -129.8
episode:  687 score: -95.1 average score -129.5
episode:  688 score: -121.5 average score -129.2
episode:  689 score: -114.3 average score -129.2
episode:  690 score: -26.2 average score -127.9
episode:  691 score: -145.7 average score -128.3
episode:  692 score: -115.5 average score -128.3
episode:  693 score: -122.2 average score -128.3
episode:  694 score: -135.1 average score -128.1
episode:  695 score: -

...

episode:  1176 score: -134.8 average score -121.8
episode:  1177 score: -90.1 average score -121.5
episode:  1178 score: -119.5 average score -122.5
episode:  1179 score: -114.2 average score -122.5
episode:  1180 score: -159.5 average score -122.3
episode:  1181 score: -91.8 average score -121.9
episode:  1182 score: -183.7 average score -122.6
episode:  1183 score: -140.0 average score -122.6
episode:  1184 score: -151.1 average score -122.5
episode:  1185 score: -102.9 average score -122.0
episode:  1186 score: -148.1 average score -122.3
episode:  1187 score: -153.5 average score -122.3
episode:  1188 score: -147.5 average score -122.1
episode:  1189 score: -84.9 average score -121.7
episode:  1190 score: -104.8 average score -121.2
episode:  1191 score: -123.0 average score -121.2
episode:  1192 score: -96.8 average score -121.7
episode:  1193 score: -152.8 average score -122.1
episode:  1194 score: 4.8 average score -122.2
episode:  1195 score: -115.3 average score -121.9
episode

episode:  1341 score: -106.4 average score -127.6
episode:  1342 score: -153.9 average score -127.8
episode:  1343 score: -145.1 average score -128.2
episode:  1344 score: -167.8 average score -128.6
episode:  1345 score: -134.5 average score -128.4
episode:  1346 score: -142.6 average score -128.8
episode:  1347 score: -156.6 average score -128.3
episode:  1348 score: -205.7 average score -128.8
episode:  1349 score: -148.2 average score -128.9
episode:  1350 score: -107.9 average score -128.3
episode:  1351 score: 4.7 average score -127.4
episode:  1352 score: -121.2 average score -127.6
episode:  1353 score: -122.3 average score -127.9
episode:  1354 score: -116.6 average score -127.7
episode:  1355 score: -116.2 average score -127.5
episode:  1356 score: -8.0 average score -126.2
episode:  1357 score: -126.6 average score -126.5
episode:  1358 score: -113.6 average score -126.3
episode:  1359 score: -115.4 average score -127.4
episode:  1360 score: -154.8 average score -127.9
episo

episode:  1506 score: -138.8 average score -132.0
episode:  1507 score: -181.9 average score -132.4
episode:  1508 score: -155.7 average score -132.6
episode:  1509 score: -141.7 average score -132.6
episode:  1510 score: -149.0 average score -132.8
episode:  1511 score: -141.2 average score -133.2
episode:  1512 score: -161.9 average score -133.6
episode:  1513 score: -160.9 average score -133.0
episode:  1514 score: -154.0 average score -132.8
episode:  1515 score: -139.9 average score -132.8
episode:  1516 score: -127.6 average score -132.4
episode:  1517 score: -116.1 average score -132.4
episode:  1518 score: -132.3 average score -132.3
episode:  1519 score: -128.5 average score -132.3
episode:  1520 score: -125.4 average score -132.2
episode:  1521 score: -143.3 average score -132.4
episode:  1522 score: -150.2 average score -132.0
episode:  1523 score: -75.5 average score -131.4
episode:  1524 score: -104.1 average score -131.0
episode:  1525 score: -138.5 average score -130.8
e

episode:  1671 score: -93.4 average score -125.0
episode:  1672 score: -184.7 average score -125.4
episode:  1673 score: -81.5 average score -124.9
episode:  1674 score: -167.1 average score -125.2
episode:  1675 score: -30.0 average score -123.7
episode:  1676 score: -132.8 average score -124.1
episode:  1677 score: -118.8 average score -124.1
episode:  1678 score: -123.3 average score -124.0
episode:  1679 score: -139.3 average score -124.0
episode:  1680 score: -143.5 average score -123.9
episode:  1681 score: -180.5 average score -123.9
episode:  1682 score: -130.2 average score -123.9
episode:  1683 score: -131.0 average score -123.7
episode:  1684 score: -120.0 average score -123.8
episode:  1685 score: -118.2 average score -124.0
episode:  1686 score: -142.3 average score -123.6
episode:  1687 score: 0.1 average score -122.6
episode:  1688 score: -129.1 average score -122.4
episode:  1689 score: -157.2 average score -123.1
episode:  1690 score: -127.9 average score -122.9
episod

episode:  1836 score: -146.5 average score -132.4
episode:  1837 score: -126.5 average score -132.5
episode:  1838 score: -127.8 average score -134.0
episode:  1839 score: -147.7 average score -134.3
episode:  1840 score: -150.2 average score -134.9
episode:  1841 score: -26.2 average score -133.8
episode:  1842 score: -92.7 average score -133.2
episode:  1843 score: -91.2 average score -132.6
episode:  1844 score: -122.1 average score -132.5
episode:  1845 score: -138.3 average score -132.9
episode:  1846 score: -128.7 average score -131.4
episode:  1847 score: -138.6 average score -132.0
episode:  1848 score: -177.0 average score -132.5
episode:  1849 score: -114.3 average score -132.2
episode:  1850 score: -127.8 average score -132.1
episode:  1851 score: -131.7 average score -132.2
episode:  1852 score: -197.9 average score -132.5
episode:  1853 score: -134.0 average score -132.7
episode:  1854 score: -137.8 average score -132.9
episode:  1855 score: -160.6 average score -133.0
epi

In [18]:
    num_episodes = 200000

    for i in range(num_episodes):
        done = False
        score = 0
        observation = env.reset()
        while not done:
            action = agent.choose_action(observation)
            observation_, reward, done, info = env.step(action)
            agent.store_transition(observation, action, reward)
            observation = observation_
            score += reward
        score_history.append(score)

        agent.learn()
        avg_score = np.mean(score_history[-100:])
        print('episode: ', i,'score: %.1f' % score,
            'average score %.1f' % avg_score)

episode:  0 score: -103.5 average score -135.5
episode:  1 score: -131.9 average score -135.5
episode:  2 score: -81.6 average score -135.2
episode:  3 score: -154.1 average score -135.3
episode:  4 score: -147.6 average score -135.4
episode:  5 score: -159.7 average score -135.8
episode:  6 score: -125.1 average score -135.8
episode:  7 score: -129.3 average score -136.9
episode:  8 score: -125.7 average score -137.3
episode:  9 score: -117.0 average score -136.5
episode:  10 score: -136.3 average score -136.1
episode:  11 score: -130.8 average score -135.8
episode:  12 score: -14.6 average score -134.2
episode:  13 score: -139.8 average score -134.1
episode:  14 score: -101.8 average score -133.9
episode:  15 score: -117.5 average score -133.7
episode:  16 score: -152.1 average score -133.9
episode:  17 score: -127.2 average score -134.2
episode:  18 score: -165.4 average score -135.4
episode:  19 score: -138.6 average score -135.4
episode:  20 score: -206.2 average score -136.1
epis

episode:  170 score: -119.9 average score -138.0
episode:  171 score: 6.2 average score -136.3
episode:  172 score: -117.5 average score -136.0
episode:  173 score: -108.6 average score -135.6
episode:  174 score: -99.0 average score -135.3
episode:  175 score: -178.0 average score -135.6
episode:  176 score: -118.3 average score -135.0
episode:  177 score: -141.6 average score -134.8
episode:  178 score: 41.4 average score -133.0
episode:  179 score: -122.1 average score -133.0
episode:  180 score: -16.8 average score -131.6
episode:  181 score: -153.0 average score -131.8
episode:  182 score: -125.4 average score -131.8
episode:  183 score: -141.7 average score -131.7
episode:  184 score: -153.4 average score -131.9
episode:  185 score: -159.3 average score -131.9
episode:  186 score: -120.7 average score -131.7
episode:  187 score: -133.5 average score -131.9
episode:  188 score: 14.6 average score -130.4
episode:  189 score: -134.5 average score -130.9
episode:  190 score: -138.2 a

episode:  338 score: -98.7 average score -127.0
episode:  339 score: -123.6 average score -127.0
episode:  340 score: -94.2 average score -126.4
episode:  341 score: -134.4 average score -126.1
episode:  342 score: -155.9 average score -126.4
episode:  343 score: -99.7 average score -125.7
episode:  344 score: -123.9 average score -125.3
episode:  345 score: -146.0 average score -125.3
episode:  346 score: -138.9 average score -125.3
episode:  347 score: -131.2 average score -125.0
episode:  348 score: -115.6 average score -124.6
episode:  349 score: -132.6 average score -124.5
episode:  350 score: -133.6 average score -124.4
episode:  351 score: -135.4 average score -124.5
episode:  352 score: -138.9 average score -124.7
episode:  353 score: -97.1 average score -124.2
episode:  354 score: -3.1 average score -123.1
episode:  355 score: -98.9 average score -122.2
episode:  356 score: -109.3 average score -122.2
episode:  357 score: -124.3 average score -122.0
episode:  358 score: -160.3

episode:  506 score: -145.0 average score -133.6
episode:  507 score: -146.3 average score -134.1
episode:  508 score: -144.9 average score -134.8
episode:  509 score: -219.2 average score -135.4
episode:  510 score: -105.4 average score -134.9
episode:  511 score: -195.3 average score -135.5
episode:  512 score: -163.4 average score -136.8
episode:  513 score: -132.7 average score -137.0
episode:  514 score: -158.5 average score -137.6
episode:  515 score: -140.9 average score -137.7
episode:  516 score: -131.3 average score -137.6
episode:  517 score: -131.4 average score -137.7
episode:  518 score: -113.3 average score -137.8
episode:  519 score: -148.4 average score -137.7
episode:  520 score: -123.3 average score -137.5
episode:  521 score: -126.7 average score -137.4
episode:  522 score: -111.5 average score -137.7
episode:  523 score: -94.7 average score -137.0
episode:  524 score: -138.2 average score -136.7
episode:  525 score: -95.6 average score -136.3
episode:  526 score: -

episode:  674 score: -140.6 average score -133.7
episode:  675 score: -168.0 average score -133.7
episode:  676 score: -155.0 average score -133.8
episode:  677 score: -184.4 average score -133.8
episode:  678 score: -108.3 average score -133.6
episode:  679 score: -137.6 average score -133.1
episode:  680 score: -130.3 average score -133.2
episode:  681 score: -135.5 average score -133.2
episode:  682 score: -134.2 average score -133.5
episode:  683 score: -95.0 average score -134.3
episode:  684 score: -183.6 average score -134.7
episode:  685 score: -134.8 average score -134.7
episode:  686 score: -132.1 average score -134.7
episode:  687 score: -126.7 average score -134.2
episode:  688 score: -124.0 average score -134.5
episode:  689 score: -87.1 average score -134.0
episode:  690 score: -195.3 average score -134.5
episode:  691 score: -128.4 average score -134.5
episode:  692 score: -133.3 average score -134.5
episode:  693 score: -135.6 average score -134.7
episode:  694 score: -

...

episode:  6290 score: -140.1 average score -136.4
episode:  6291 score: -165.4 average score -136.6
episode:  6292 score: -115.8 average score -136.4
episode:  6293 score: -145.7 average score -137.1
episode:  6294 score: -139.3 average score -137.0
episode:  6295 score: -137.5 average score -137.1
episode:  6296 score: -166.9 average score -137.3
episode:  6297 score: -148.1 average score -137.5
episode:  6298 score: -141.6 average score -138.1
episode:  6299 score: -185.4 average score -139.0
episode:  6300 score: -110.4 average score -138.7
episode:  6301 score: -117.0 average score -138.6
episode:  6302 score: -145.2 average score -138.8
episode:  6303 score: -304.2 average score -140.6
episode:  6304 score: -151.0 average score -140.3
episode:  6305 score: -93.1 average score -139.7
episode:  6306 score: -145.3 average score -139.9
episode:  6307 score: -136.8 average score -139.3
episode:  6308 score: -151.2 average score -139.6
episode:  6309 score: -201.6 average score -140.4
e

episode:  6455 score: -105.8 average score -129.8
episode:  6456 score: -95.8 average score -129.7
episode:  6457 score: -107.8 average score -129.8
episode:  6458 score: -43.0 average score -128.6
episode:  6459 score: -137.9 average score -128.4
episode:  6460 score: -146.1 average score -128.4
episode:  6461 score: -184.6 average score -128.8
episode:  6462 score: -156.4 average score -129.1
episode:  6463 score: -178.3 average score -129.3
episode:  6464 score: -135.8 average score -130.4
episode:  6465 score: -137.6 average score -130.3
episode:  6466 score: -141.2 average score -130.7
episode:  6467 score: -110.8 average score -130.3
episode:  6468 score: -139.7 average score -131.6
episode:  6469 score: -126.5 average score -131.4
episode:  6470 score: -203.0 average score -133.4
episode:  6471 score: -112.7 average score -133.0
episode:  6472 score: -117.2 average score -132.7
episode:  6473 score: -143.7 average score -132.9
episode:  6474 score: -177.9 average score -134.1
ep

episode:  6620 score: -176.5 average score -139.0
episode:  6621 score: -105.1 average score -140.3
episode:  6622 score: -113.9 average score -140.2
episode:  6623 score: -296.7 average score -141.5
episode:  6624 score: -138.6 average score -141.4
episode:  6625 score: -131.2 average score -141.2
episode:  6626 score: -126.3 average score -141.4
episode:  6627 score: -129.2 average score -141.2
episode:  6628 score: -161.1 average score -141.4
episode:  6629 score: -119.5 average score -141.3
episode:  6630 score: -118.7 average score -141.1
episode:  6631 score: -122.2 average score -141.2
episode:  6632 score: -152.0 average score -141.7
episode:  6633 score: 30.0 average score -139.8
episode:  6634 score: -112.5 average score -139.8
episode:  6635 score: -152.8 average score -140.1
episode:  6636 score: -142.4 average score -137.9
episode:  6637 score: -128.5 average score -137.8
episode:  6638 score: -132.1 average score -137.8
episode:  6639 score: -10.7 average score -136.4
epi

episode:  6785 score: -144.7 average score -134.8
episode:  6786 score: -209.5 average score -135.8
episode:  6787 score: -151.2 average score -136.4
episode:  6788 score: -110.6 average score -136.2
episode:  6789 score: -138.1 average score -136.4
episode:  6790 score: -133.0 average score -138.1
episode:  6791 score: -112.9 average score -137.9
episode:  6792 score: -146.5 average score -137.9
episode:  6793 score: -158.8 average score -138.0
episode:  6794 score: -122.8 average score -137.9
episode:  6795 score: -117.9 average score -137.1
episode:  6796 score: -179.8 average score -137.3
episode:  6797 score: -96.1 average score -137.4
episode:  6798 score: -115.9 average score -137.7
episode:  6799 score: 15.9 average score -136.2
episode:  6800 score: -139.0 average score -136.3
episode:  6801 score: 24.3 average score -134.6
episode:  6802 score: -140.5 average score -135.0
episode:  6803 score: -130.7 average score -134.9
episode:  6804 score: -142.4 average score -135.1
episo

KeyboardInterrupt: 